In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import os
from shapely.geometry import box
from shapely.geometry import Polygon, Point
import plotly.express as px


In [49]:
epsg = 32614 # UTM-14

## CDMX Data

In [50]:
cdmx = gpd.read_file('../data/cdmx/Polygons.gpkg', layer='cdmx')
cdmx = cdmx.to_crs(epsg=4326).to_crs(epsg=epsg)

cdmx_bbox = cdmx.total_bounds
cdmx_bbox_geometry = box(*cdmx_bbox)

## Catalog

In [51]:
stations = pd.read_csv('../data/cdmx/cat_estacion.csv', encoding='latin1', skiprows=1)
stations = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.longitud, stations.latitud))
stations = stations.set_crs(epsg=4326).to_crs(epsg=epsg)
stations['obs_estac'] = stations['obs_estac'].fillna('')
stations['obs_estac'] = stations['obs_estac'].apply(lambda x: 'Activa' if 'Fin' not in x else 'Inactiva')


# STATIONS

In [52]:
dataset = pd.read_csv('../data/datasets/ds_rama2.csv')

# Identify all stations and pollutants present in the dataset
stations_rama = {col: {'pollutants' : dataset[['pollutant', col]]
                   .groupby('pollutant')
                   .count().query(f'{col} > 0')
                   .index.tolist(),
                   'min_date' : dataset.query(f'{col}.notna()').date.min(),
                   'max_date': dataset.query(f'{col}.notna()').date.max()
                   }
                   for col in dataset.columns if col not in ['pollutant', 'date','month','year']}

# Remove stations with no pollutants
stations_rama = {k: v for k, v in stations_rama.items() if len(v) > 0}

# Add pollutants to stations
stations['pollutants'] = stations['cve_estac'].apply(lambda x: stations_rama.get(x, {'pollutants':np.nan}).get('pollutants'))
stations['first_date'] = stations['cve_estac'].apply(lambda x: stations_rama.get(x, {'min_date':np.nan}).get('min_date'))
stations['last_date'] = stations['cve_estac'].apply(lambda x: stations_rama.get(x, {'max_date':np.nan}).get('max_date'))

In [53]:
bbox_gdf = gpd.GeoDataFrame({'geometry': [cdmx_bbox_geometry]}, crs=epsg)
m = bbox_gdf.explore(color='red')#, alpha=0.5)
stations.explore(m = m, column = 'obs_estac', marker_kwds={'radius': 5}, cmap='viridis', legend=True, vmin=0.2, vmax=0.8)

### Filter stations
Filter stations by thoise who have pollutants regsters, inside the bounding box and active state

In [54]:
# Get only stations with pollutants and within the CDMX bbox
stations.query('pollutants.notna() and obs_estac == "Activa"', engine='python', inplace=True)
stations = stations.clip(cdmx_bbox_geometry)

In [55]:
stations.drop(columns=['geometry']).to_csv('../data/cdmx/estaciones.csv', index=False)

In [8]:
m = bbox_gdf.explore(color='red')#, alpha=0.0)
stations.explore(m = m, column = 'cve_estac', marker_kwds={'radius': 5}, cmap='viridis', legend=True, vmin=0.2, vmax=0.8)

# Create grid

In [9]:
# Get the top-left coordinates in meters
minx, maxy = bbox_gdf.bounds.iloc[0][['minx', 'maxy']].values
maxx, miny = bbox_gdf.bounds.iloc[0][['maxx', 'miny']].values

resolution  = 500 

# Create array from top-left coordinates spacing by 500 meters
x = np.arange(minx, maxx, resolution )
y = np.arange(miny, maxy, resolution )

# Create a meshgrid
X, Y = np.meshgrid(x, y)

st = 9
dx = 8
dy = 1
X = X[0+dx::st, 0+dx::st]
Y = Y[0+dy::st, 0+dy::st]

In [10]:
#Geodataframe from meshgrid
grid = gpd.GeoDataFrame(geometry=[box(x - resolution/2, y - resolution/2, x + resolution/2 , y + resolution/2 ) for x, y in zip(X.ravel(), Y.ravel())], crs=epsg)

In [11]:
stations_quantized = stations[['cve_estac','geometry']].copy()

# Modify the geometry
stations_quantized['geometry'] = stations_quantized['geometry'].apply(
    lambda geom: box(
        ((geom.x - (minx + dx*resolution)) // (st * resolution)) * (st * resolution) + (minx + dx*resolution),
        ((geom.y - (miny + dy*resolution)) // (st * resolution)) * (st * resolution) + (miny + dy*resolution),
        ((geom.x - (minx + dx*resolution)) // (st * resolution)) * (st * resolution) + (minx + dx*resolution) + (st * resolution),
        ((geom.y - (miny + dy*resolution)) // (st * resolution)) * (st * resolution) + (miny + dy*resolution) + (st * resolution)
    )
)

In [12]:
fig = grid.explore(color='black', style_kwds={'fillOpacity': 0})
cdmx.explore(m=fig, color='red',style_kwds={'fill': False})
stations_quantized.explore(m=fig, column='cve_estac', style_kwds={'fillOpacity': 0.25})
stations.explore(m=fig, column='cve_estac',marker_kwds={'radius': 5}, style_kwds={'fillOpacity': 1})

In [17]:
# sorted(stations.cve_estac.unique())

## Find Optimal Grid

In [13]:
# Get the top-left coordinates in meters
minx, maxy = bbox_gdf.bounds.iloc[0][['minx', 'maxy']].values
maxx, miny = bbox_gdf.bounds.iloc[0][['maxx', 'miny']].values

resolution  = 500 

# Create array from top-left coordinates spacing by 500 meters
x = np.arange(minx, maxx, resolution )
y = np.arange(miny, maxy, resolution )

# Create a meshgrid
X, Y = np.meshgrid(x, y)

stations_temp = stations.set_index('cve_estac')[['geometry']].copy()

In [14]:
def sum_distances(row):
    point = row["geometry"]  # Punto en stations_temp
    polygon = row["geometry_poly"]  # Polígono en stations_quantized

    if polygon and point:
        vertices = list(polygon.exterior.coords)  # Extraer coordenadas de los vértices
        distances = [Point(v).distance(point) for v in vertices]  # Calcular distancias
        return sum(distances)  # Sumar todas las distancias
    return None  # En caso de geometría inválida


In [15]:
records = []

for st in range(1, min(X.shape)//2):
    for dx in range(0, st):
        for dy in range(0, st):
            Xs = X[0+dx::st, 0+dx::st]
            Ys = Y[0+dy::st, 0+dy::st]

            stations_quantized = stations[['cve_estac','geometry']].copy()

            # Modify the geometry
            stations_quantized['geometry'] = stations_quantized['geometry'].apply(
                lambda geom: box(
                    ((geom.x - (minx + dx*resolution)) // (st * resolution)) * (st * resolution) + (minx + dx*resolution),
                    ((geom.y - (miny + dy*resolution)) // (st * resolution)) * (st * resolution) + (miny + dy*resolution),
                    ((geom.x - (minx + dx*resolution)) // (st * resolution)) * (st * resolution) + (minx + dx*resolution) + (st * resolution),
                    ((geom.y - (miny + dy*resolution)) // (st * resolution)) * (st * resolution) + (miny + dy*resolution) + (st * resolution)
                )
            )
            stations_quantized.set_index('cve_estac', inplace=True)
            
            # Crear un nuevo GeoDataFrame combinando ambos
            merged_gdf = stations_temp.copy()
            merged_gdf["geometry_poly"] = stations_quantized["geometry"]

            # Aplicar la función a cada fila
            merged_gdf["sum_distances"] = merged_gdf.apply(sum_distances, axis=1)
            
            cost = merged_gdf["sum_distances"].sum()
            
            num_cells = Xs.shape[0] * Xs.shape[1]
            
            records.append({'st': st, 'dx': dx, 'dy': dy, 'cost': cost, 'num_cells': num_cells})
    

df = pd.DataFrame(records)

In [16]:
def pareto_front(df, objectives):
    """
    Obtiene las soluciones no dominadas (frente de Pareto).
    :param df: DataFrame de pandas con las soluciones.
    :param objectives: Lista con los nombres de las columnas objetivo (minimización o maximización).
    :return: DataFrame con las soluciones no dominadas.
    """
    # Convertir a numpy para eficiencia
    data = df[objectives].values
    num_points = data.shape[0]
    is_pareto = np.ones(num_points, dtype=bool)
    
    for i in range(num_points):
        if is_pareto[i]:
            is_pareto[is_pareto] = np.any(data[is_pareto] < data[i], axis=1)  # Dominancia estricta
            is_pareto[i] = True  # Mantener el punto actual
    
    return df[is_pareto]

In [ ]:

fig = px.scatter(
    title='Cost vs Number of Cells',
    labels={'num_cells': 'Number of Cells', 'cost': 'Cost', 'st': 'Step'}
)


df_min_cost = pareto_front(df, ['num_cells', 'cost'])
df_min_cost.join(df.set_index(['num_cells', 'cost'])[['st','dx','dy']], on=['num_cells', 'cost'], lsuffix='_min', rsuffix='_all')

fig.add_scatter(
    x=df_min_cost['num_cells'], 
    y=df_min_cost['cost'], 
    mode='lines+markers', 
    name='Min Cost'
    , line=dict(color='white'),
    marker=dict(size=10),
    hovertext=df_min_cost[['st', 'dx', 'dy']].apply(lambda row: f"st={row['st']}, dx={row['dx']}, dy={row['dy']}", axis=1)
)

fig.add_scatter(
    x=df['num_cells'], 
    y=df['cost'], 
    mode='markers', 
    name='Data Points',
    hovertext=df[['st', 'dx', 'dy']].apply(lambda row: f"st={row['st']}, dx={row['dx']}, dy={row['dy']}", axis=1),
    marker=dict(size=7, color=df['st'], colorscale='Viridis', showscale=True)
)

fig.update_layout(
    width=800, 
    height=800, 
    legend=dict(
        x=1, 
        y=1.05, 
        traceorder="normal", 
        font=dict(size=10)
    ),
    template='plotly_dark',
    paper_bgcolor='rgba(0,0,0,0)', 
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis_title="Number of Cells",
    yaxis_title="Cost"
)

fig.show()